In [1]:
# Reference to ../src
import os
import sys
module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import csv
import os

from dataset.MicroscopyTrainDataLoader import MicroscopyTrainDataLoader
from experiments.microscopy.microscopy import experiment, get_model
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
from sklearn.metrics import classification_report

import numpy as np; np.random.seed(0)
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt

In [2]:
OUTPUT_PATH = '../labels/microscopy.csv'
JSON_INPUT_PATH = "../src/experiments/microscopy/shallow-resnet50.json"
JSON_OUTPUT_PATH = "../src/experiments/microscopy/out_shallow_resnet50.json"

In [3]:
import json

with open(JSON_INPUT_PATH) as json_file:
    models = json.load(json_file)

models

{'resnet50.layer4-2': {'id': 'serene-brook-273', 'layers': [3, 4, 6, 3]},
 'resnet50.layer4-1': {'id': 'clean-grass-271', 'layers': [3, 4, 6, 2]},
 'resnet50.layer4-0': {'id': 'apricot-tree-269', 'layers': [3, 4, 6, 1]},
 'resnet50.layer3-5': {'id': 'cerulean-cosmos-267', 'layers': [3, 4, 6, 0]},
 'resnet50.layer3-4': {'id': 'summer-fog-265', 'layers': [3, 4, 5, 0]},
 'resnet50.layer3-3': {'id': 'toasty-universe-263', 'layers': [3, 4, 4, 0]},
 'resnet50.layer3-2': {'id': 'proud-breeze-261', 'layers': [3, 4, 3, 0]},
 'resnet50.layer3-1': {'id': 'fragrant-glitter-258', 'layers': [3, 4, 2, 0]},
 'resnet50.layer3-0': {'id': 'tough-wave-256', 'layers': [3, 4, 1, 0]},
 'resnet50.layer2-3': {'id': 'crisp-shape-254', 'layers': [3, 4, 0, 0]},
 'resnet50.layer2-2': {'id': 'magic-meadow-252', 'layers': [3, 3, 0, 0]},
 'resnet50.layer2-1': {'id': 'comic-tree-250', 'layers': [3, 2, 0, 0]},
 'resnet50.layer2-0': {'id': 'unique-lion-247', 'layers': [3, 1, 0, 0]},
 'resnet50.layer1-2': {'id': 'wobbly-

In [4]:
def load_shallow_model(model_id, model_dict):
    model_name, experiment_name = model_id.split('.')
    model = get_model(model_name, "shallow", 4, layers=model_dict[model_id]['layers'], pretrained=True)
    
    checkpoint = torch.load('../outputs/{0}/checkpoint.pt'.format(model_dict[model_id]['id']))
    model.load_state_dict(checkpoint)
    
    return model

In [5]:
resnet50_4_2 = load_shallow_model('resnet50.layer4-2', models)
resnet50_4_1 = load_shallow_model('resnet50.layer4-1', models)
resnet50_4_0 = load_shallow_model('resnet50.layer4-0', models)
resnet50_3_5 = load_shallow_model('resnet50.layer3-5', models)
resnet50_3_4 = load_shallow_model('resnet50.layer3-4', models)
resnet50_3_3 = load_shallow_model('resnet50.layer3-3', models)

In [6]:
import torch.nn.functional as F

class MyEnsemble(nn.Module):
    def __init__(self, model1, model2, model3, model4, model5, model6, nb_classes=4):
        super(MyEnsemble, self).__init__()
        self.model1 = model1
        self.model2 = model2
        self.model3 = model3
        self.model4 = model4
        self.model5 = model5
        self.model6 = model6
        # Remove last linear layer
        self.model1.fc = nn.Identity()
        self.model2.fc = nn.Identity()
        self.model3.fc = nn.Identity()
        self.model4.fc = nn.Identity()
        self.model5.fc = nn.Identity()
        self.model6.fc = nn.Identity()
        
        # Create new classifier
        self.classifier = nn.Linear(2048+2048+2048+1024+1024+1024, nb_classes)
        
    def forward(self, x):
        x1 = self.model1(x.clone())  # clone to make sure x is not changed by inplace methods
        x1 = x1.view(x1.size(0), -1)
        x2 = self.model2(x)
        x2 = x2.view(x2.size(0), -1)
        x3 = self.model3(x)
        x3 = x3.view(x3.size(0), -1)
        x4 = self.model4(x)
        x4 = x4.view(x4.size(0), -1)
        x5 = self.model5(x)
        x5 = x5.view(x5.size(0), -1)
        x6 = self.model6(x)
        x6 = x6.view(x6.size(0), -1)
        
        x = torch.cat((x1, x2, x3, x4, x5, x6), dim=1)
        
        x = self.classifier(F.relu(x))
        return x

In [7]:
ensemble = MyEnsemble(resnet50_4_2, resnet50_4_1, resnet50_4_0, resnet50_3_5, resnet50_3_4, resnet50_3_3)

In [8]:
checkpoint = torch.load('/workspace/outputs/{0}/checkpoint.pt'.format('prime-river-402'))
ensemble.load_state_dict(checkpoint)

<All keys matched successfully>

In [9]:
OUTPUT_PATH = '../labels/microscopy.csv'
loader = MicroscopyTrainDataLoader(OUTPUT_PATH)
test_dataset = loader.get_test_dataset()

In [10]:
def test(model):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    criterion = nn.CrossEntropyLoss()
    
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())
    model = model.to(device)
    
    model.eval()
    
    kwargs = {'num_workers': 16, 'pin_memory': True} if torch.cuda.is_available() else {}
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, **kwargs)
    
    test_loss, n_iter = 0, 0
    total, correct = 0, 0
    
    y_true = []
    y_pred = []
    img_names = []
    
    for images, labels, img_path in test_loader:         
        y_true += labels
        img_names += img_path

        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        y_pred += predicted.cpu()

        loss = criterion(outputs, labels)

        total += labels.size(0)
        correct += torch.sum(predicted == labels.data)
        test_loss += loss.item()
        n_iter += 1

    acc = 100 * torch.true_divide(correct, total)
    average_loss = test_loss / n_iter
    
    target_names = test_dataset.codec.classes_
    print(classification_report(y_true, y_pred, target_names=target_names, digits=4))
    
    return acc.cpu(), average_loss, y_true, y_pred, img_names, trainable_params, total_params

In [11]:
acc, avg_loss, y_true, y_pred, img_names, trainable_params, total_params = test(ensemble)

              precision    recall  f1-score   support

        DMEL     0.6452    0.4545    0.5333        88
        DMFL     0.9371    0.9437    0.9404       284
        DMLI     0.9363    0.9802    0.9578       405
        DMTR     0.7624    0.8021    0.7817        96

    accuracy                         0.8958       873
   macro avg     0.8202    0.7951    0.8033       873
weighted avg     0.8881    0.8958    0.8900       873



In [12]:
from sklearn.metrics import confusion_matrix
import numpy as np

cm = confusion_matrix(y_true, y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
cm.diagonal()

array([0.45454545, 0.94366197, 0.98024691, 0.80208333])

NameError: name 'cm' is not defined